In [104]:
#http imports
import requests
from bs4 import BeautifulSoup
from datetime import date
import csv
import os
import json
import urllib.request
import pandas as pd
from datetime import date
from datetime import datetime

# pass headers in a dict to the headers parameter to mimic a real human doing the request using a web browser
headers = {
    'accept': '*/*',
    'content-type': 'text/html;charset=UTF-8',
    'content-encoding': 'gzip',
    'origin': 'www.zillow.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'
}

In [ ]:
"""
#selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

filepath = "D:\\Daniel Python Project\\chromedriver_win32"
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.set_window_size(1120, 1000)

url = "https://us.kowtowclothing.com/collections/all"
driver.get(url)
classForImage = "grid-product-link-wrapper"
first_search_bar = driver.find_element_by_class_name(classForImage)
print(first_search_bar)
"""

In [99]:
num_clothing = 25 #how many products to scrape
url = "https://us.kowtowclothing.com/collections/all"

#init details
brand = []
product_type = [] #name extracted from url (can be blank)
product_name = []
product_description = []
price = [] #price before sale (can be blank)
sales_price = [] #price customer pays
color = []
material = []
product_url = []
picture_url = []
timestamp = []
gender = [] #website did not have gender specified so all unisex
new_arrival = []

current_date = date.today()

#opening url
response = requests.get(url, headers=headers)

#reading content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

products_scraped = 0

product_link_tags = soup.find_all("a", {'class':'grid-product-link-wrapper'})
product_links = []
for tag in product_link_tags:
    product_links.append('https://us.kowtowclothing.com' + tag.get('href'))
    
"""
#dupe checker
found_links = []
dupes = 0
for link in product_links:
    if link in found_links:
        dupes += 1
    found_links.append(link)
print("FOUND", dupes, "DUPLICATE LINKS")
"""
#print(product_links) #only for first page currently (look at button)

['https://us.kowtowclothing.com/collections/all/products/quilt-vest-green-and-white-stripe', 'https://us.kowtowclothing.com/collections/all/products/sculptured-wrap-top-green-and-white-stripe', 'https://us.kowtowclothing.com/collections/all/products/sculptured-dress-green-and-white-stripe', 'https://us.kowtowclothing.com/collections/all/products/quilt-bag-green-and-white-stripe', 'https://us.kowtowclothing.com/collections/all/products/playground-shirt-peony', 'https://us.kowtowclothing.com/collections/all/products/quilt-bag-peony', 'https://us.kowtowclothing.com/collections/all/products/culottes-peony', 'https://us.kowtowclothing.com/collections/all/products/sculptured-dress-ink', 'https://us.kowtowclothing.com/collections/all/products/building-block-cap-sleeve-dress-ink', 'https://us.kowtowclothing.com/collections/all/products/study-pant-ink', 'https://us.kowtowclothing.com/collections/all/products/sculptured-wrap-top-ink', 'https://us.kowtowclothing.com/collections/all/products/henri

In [105]:
count = 0
while products_scraped < num_clothing and count < len(product_links): #mouse pog
    product_page = BeautifulSoup(requests.get(product_links[count], headers=headers).content, 'html.parser')
    
    product_div = product_page.find('div', {'itemtype': 'http://schema.org/Product'}) 
    
    #urls, desc, and brand
    product_metas = product_div.find_all('meta')
    """
    0: product url
    1: description
    2: image url
    3: brand
    ...: not needed
    """
    product_url.append(product_metas[0].get('content'))
    product_description.append(product_metas[1].get('content'))
    picture_url.append(product_metas[2].get('content'))
    brand.append(product_metas[3].get('content'))
    
    
    #money and color
    product_misc = product_div.find_all('h2', {'class':'subtitle is-9'})
    product_misc_text = []
    for h2 in product_misc:
        product_misc_text.append(h2.get_text()) #0 and 1 price with \n, 3 is color
    #splitting price
    money_split = product_misc_text[0].split('\n') #0 is '', 1,2 are prices
    price.append(money_split[1])
    sales_price.append(money_split[2])
    color.append(product_misc_text[2])
    
    #product name
    name = product_div.find('h1', {'itemprop':'name'})
    product_name.append(name.get_text())
    
    #material
    #extracting material using product description THIS CAN BE REWORKED LOOK FOR DIV CLASS = swatch js-swatch ON PRODUCT PAGE
    materials = []
    material_tags = product_div.find_all('tr')
    
    for tag in material_tags:
        tag_text = tag.find('td').get_text()
        split_text = tag_text.split(':')[1]
        if 'cotton' in split_text and split_text != 'cotton': #handling duplicate cotton entries (organic-, elastane-, -tencel)
            cotton_split = split_text.split('-')
            print(split_text)
            if cotton_split[0] == 'cotton':
                materials.append(cotton_split[1])
            else:
                materials.append(cotton_split[0])
            
        elif not 'fit' in tag_text and not 'model' in tag_text and not 'trim' in tag_text and not 'process' in tag_text and not 'content' in tag_text:
            if '-' in split_text: #handling for multi-word materials that are not cotton
                new_word = ''
                split_word = split_text.split('-')
                for word in split_word:
                    new_word += word
                    if word != split_word[-1]:
                        new_word += ' '
                materials.append(new_word)
            else:
                materials.append(split_text)
    
    desc = product_metas[1].get('content') #product description from earlier
    material_list = [] #all materials / processes in product
    
    #adding materials to list
    for mat in materials:
        if mat in desc:
            material_list.append(mat)

    #recombining materials into one string
    combined_material = ''
    non_cotton = ''
    for mat in material_list: #unfinished
        if mat == 'organic':
            combined_material = 'organic cotton'
        elif mat == 'elastane':
            combined_material = 'elastane cotton'
        elif mat == 'tencel':
            combined_material = 'cotton tencel'
        elif mat != 'cotton':
            non_cotton = mat
    combined_material = non_cotton + combined_material
    
    material.append(combined_material)
    
    #gender and timestamp 
    gender.append('unisex') #website does not have gendered clothing 
    timestamp.append(datetime.now())
    
    count += 1

organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tencel
organic-cotton
elastane-cotton
cotton-tencel
cotton-tenc